In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer,execute, transpile, assemble
from qiskit.circuit.library.standard_gates import RXGate, RYGate, MCXGate
from qiskit.quantum_info import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.circuit import Parameter, ParameterVector

from scipy.optimize import minimize
from scipy.stats import entropy
import random

from scipy.special import rel_entr
import yfinance as yf
import pandas as pd
from collections import defaultdict
import time

In [ ]:
def float_range(num, spacing):
    arr = np.arange(-spacing*(num+1)/2,spacing*(num+1)/2, spacing)
    return arr

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Downloading data set
Ticker_list = ["AAPL", "MSFT", "AMZN", "GOOGL","JPM", "^GSPC", "^NDX","NVDA", "TSLA", "JNJ"]
start_date = "2022-01-01"
end_date = "2022-03-31"
data = yf.download(Ticker_list, start=start_date, end=end_date)
data = data.interpolate()
data = data.loc[:, "Adj Close"]
return_data = data.pct_change().dropna()

nq = 4
bound = float_range(2**nq, 0.01)  # create bins
stock_distributions = defaultdict(list)
# Select the stock you are interested in
# Loop over each stock
for stock_interest in Ticker_list:
    dist = np.zeros(2**nq)
    single_data = return_data[stock_interest]
    for i in range(len(single_data)):
        temp = single_data[i] 
        for j in range(2**nq):
            if temp > bound[j] and temp < bound[j+1]:
                dist[j] += 1 

    # Normalizing the distribution
    real_target = dist / sum(dist)
    
    # Add the distribution to the defaultdict
    stock_distributions[stock_interest].append(real_target)




In [ ]:
# Load the log-normal distribution
N = 1000000

# parameters for considered random distribution
S = 6.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.04  # annual interest rate of 4%
T = 90 / 365  # 40 days to maturity

mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

# Sample from the log-normal distribution
log_normal_samples = np.random.lognormal(mean=mu, sigma=sigma, size=N)

# Round samples to the nearest 0.5 within specified range
log_normal_samples = np.round(log_normal_samples * 2) / 2  # rounding to nearest 0.5
log_normal_samples = log_normal_samples[(log_normal_samples >= 0) & (log_normal_samples <= 16)]
real_target = log_normal_samples
# Calculate the probability distribution from log_normal_samples
bins = np.arange(0, 16.5, 0.5) # bins for histogram
hist, bin_edges = np.histogram(log_normal_samples, bins=bins, density=True)

# Assign the distribution to real_target
real_target = hist / np.sum(hist)

# Plot the histogram
plt.bar(bin_edges[:-1], hist, width=0.5, color='skyblue', alpha=0.7, edgecolor='black')
plt.xlabel('Spot Price at Maturity $S_T$')
plt.ylabel('Probability')
plt.title('Log-normal Distribution')
plt.show()


In [ ]:
# Parameters for the binomial distribution
n = 127  # number of trials
p = 0.3 # probability of success in each trial

# Number of samples you want to generate
num_samples = 1000000

# Sample from the binomial probability distribution
binomial_samples = np.random.binomial(n, p, size=num_samples)

# Calculate the probability distribution from binomial_samples
unique, counts = np.unique(binomial_samples, return_counts=True)
binomial_distribution = counts / num_samples


# Create a full distribution array with zeros for missing values
real_target = np.zeros(n+1)
real_target[unique] = binomial_distribution
real_target = real_target / np.sum(real_target)


# Plot the histogram
plt.hist(binomial_samples, bins=np.arange(n+2)-0.5, density=True, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Successes')
plt.ylabel('Probability')
plt.title('Binomial Distribution (n={}, p={})'.format(n, p))

# Customize the x-axis tick labels
plt.xticks(np.arange(n+1))

plt.show()